In [74]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import urllib
import os
import time

In [75]:
showdict = {}
failures = []
failuress = set()

In [90]:
def getpage(animeid):
    url = f"https://myanimelist.net/anime/{animeid}"
    with urllib.request.urlopen(url) as html:
        bs = BeautifulSoup(html, 'html.parser')
        titles = bs.find_all('title')
        if len(titles) != 1:
            return (False, animeid)

        title = str(titles[0]).split(">")[1].split("<")[-2].replace("\n", "")
        
        links = bs.find_all('link')
        found = []
        for l in links:
            if "myanimelist" in str(l) and str(animeid) in str(l):
                found.append(l)
        
        if len(found) != 1:
            return (False, animeid)
                
        bakeurl = str(found[0]).split('"')[1]+ "/characters"
        showcharacters = []
        with urllib.request.urlopen(bakeurl) as charpage:
            bs = BeautifulSoup(charpage, 'html.parser')
            links = bs.find_all('a')
            for l in links:
                if ".php" not in str(l) and "view_more" not in str(l) and "https://myanimelist.net/character" in str(l) and "myanimelist" in str(l) and "cdn.myanimelist.net" not in str(l):
                    linkdecomp = str(l).split("/")[-2].split('"')[0]
                    characterid = str(l).split("/")[4]
                    showcharacters.append((linkdecomp, characterid))

        return True, (title, showcharacters)

for animeid in tqdm(range(0, 10000)):
    if animeid in showdict.keys() or animeid in failuress:
        pass
    else:
        time.sleep(5.0)
        try:
            status, content = getpage(animeid)
            if status:
                showdict[animeid] = content
            else:
                failuress.add(animeid)
                failures.append(animeid)
        except Exception as e:
            failuress.add(animeid)
            failures.append((animeid, e))

100%|██████████| 10000/10000 [14:58:45<00:00,  5.39s/it] 


In [89]:
import pickle
failures_str = []
for f in failures:
    if isinstance(f, int):
        failures_str.append((f))
    elif isinstance(f, tuple) and len(f) == 2 and isinstance(f[0], int):
        failures_str.append((int(f[0]), str(f[1])))
    else:
        print(f)


pickle.dump((showdict, failures_str), open("MALSCmaldump_0_10000.p", "wb"))